In [7]:
# Packages that you will need to install
!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install sacremoses

^C
Traceback (most recent call last):
  File "/bin/pip", line 6, in <module>
    from pkg_resources import load_entry_point
  File "/usr/lib/python3/dist-packages/pkg_resources/__init__.py", line 36, in <module>
    import email.parser
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 844, in exec_module
  File "<frozen importlib._bootstrap_external>", line 939, in get_code
  File "<frozen importlib._bootstrap_external>", line 1037, in get_data
KeyboardInterrupt


In [1]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from torch.utils.data import DataLoader
import torch
from torch.utils.data import DataLoader, Dataset

/home/dell/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('Luo-Eng.csv')
display(df)

,Luo Content,English Content
0,"Jaode niluongo ni Maaka,",And the name of his wife [was] Maacah.
1,Giil odiechiengʼ duto nikech nyingi; gidhialo ...,"In thy name shall they rejoice all the day, an..."
2,Anakech ngʼama adwaro kecho kendo anangʼwon-ne...,"Truly, I am the Oft-Forgiving, the Most Mercif..."
3,"Negie kaka luoro dwaro negogi, nikech Nyasaye ...","But they will be overwhelmed with dread, for G..."
4,kama gichokoree (Jonjore).,and gathered at one place.
...,...,...
2825276,Tachi waza kata dori dai ikkyo omote,baking a pie or loaf of bread
2825277,kore kuom weche modonjnego.,be set on the response.
2825278,giluwe ka giwuotho gi tiendegi.,by following in His footsteps.
2825279,Kik Me - Legs Pics Kik User Me Date Ing,don't let me down - the chainsmokers ft. daya


In [4]:
total_rows = len(df)
chunk_size = total_rows // 2000

chunks = [df.iloc[i * chunk_size:(i + 1) * chunk_size] for i in range(100)]

# Name the chunk were to use
working = chunks[0]

display(working)

,Luo Content,English Content
0,"Jaode niluongo ni Maaka,",And the name of his wife [was] Maacah.
1,Giil odiechiengʼ duto nikech nyingi; gidhialo ...,"In thy name shall they rejoice all the day, an..."
2,Anakech ngʼama adwaro kecho kendo anangʼwon-ne...,"Truly, I am the Oft-Forgiving, the Most Mercif..."
3,"Negie kaka luoro dwaro negogi, nikech Nyasaye ...","But they will be overwhelmed with dread, for G..."
4,kama gichokoree (Jonjore).,and gathered at one place.
...,...,...
1407,"Kar timo kamano, to okete ewi rachungi taya, m...","Instead, he puts it on a lamp stand so that th..."
1408,"Godi ducu gibimukogi gireto woko piny, pe gibi...","If they escape into the mountains, it will be ..."
1409,Lubanga onongo omiyo lungolkop me telo jo Icra...,God had given the Israelites judges to lead th...
1410,Gima Wuoro manyo kuom joma lame gadier ok en k...,This is the worship which the Father seeks; th...


In [57]:
# Initialize the MarianMT tokenizer
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-luo-en")

# Tokenize the data with padding and truncation
tokenized_data = tokenizer(
    working['Luo Content'].tolist(),
    text_target=working['English Content'].tolist(),
    padding=True,
    truncation=True,
    return_tensors="pt"
)
display(tokenized_data)

{'input_ids': tensor([[19414,    16,  6288,  ..., 52235, 52235, 52235],
        [ 1701,  6071,   826,  ..., 52235, 52235, 52235],
        [39655,    38, 18178,  ..., 52235, 52235, 52235],
        ...,
        [ 3147,    49, 20416,  ..., 52235, 52235, 52235],
        [  101,   181, 19467,  ..., 52235, 52235, 52235],
        [  277,   163,  1211,  ..., 52235, 52235, 52235]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[  414,     5,   420,  ..., 52235, 52235, 52235],
        [  134,  6347,   650,  ..., 52235, 52235, 52235],
        [ 7400,     3,    47,  ..., 52235, 52235, 52235],
        ...,
        [  836,    12,     4,  ..., 52235, 52235, 52235],
        [  414,  1155,   656,  ..., 52235, 52235, 52235],
        [  277,   430,    45,  ..., 52235, 52235, 52235]])}

In [58]:

# Define your dataset class
class CustomDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.tokenized_data["input_ids"][idx],
            "attention_mask": self.tokenized_data["attention_mask"][idx],
            "labels": self.tokenized_data["labels"][idx],
        }

In [59]:
# Define your dataset using the tokenized data
dataset = CustomDataset(tokenized_data)

# Initialize the MarianMT model
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-luo-en")

# Define the training parameters
learning_rate = 1e-4
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
epochs = 5

# Define the DataLoader for training
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)

# Define the training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}: Average Loss - {average_loss}")


Epoch 1: Average Loss - 2.944636896252632
Epoch 2: Average Loss - 0.9944823041558266
Epoch 3: Average Loss - 0.6317375559359789
Epoch 4: Average Loss - 0.44924096539616587
Epoch 5: Average Loss - 0.3311813708394766


In [60]:
# Check the model's translation
input_sentence = " Anakech"  
tokenized_input = tokenizer(input_sentence, return_tensors="pt")
with torch.no_grad():
    model.eval()
    output = model.generate(**tokenized_input, max_length=50)
translated_sentence = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Input Sentence (Luo): {input_sentence}")
print(f"Translated Sentence (English): {translated_sentence}")

Input Sentence (Luo):  gichokoree
Translated Sentence (English): and gathered at the meeting


In [61]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Define the path to save the model
save_path = os.path.join(current_directory, "Luo-Eng-Translation-Model")

# Save the trained model
model.save_pretrained(save_path)


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[52235]], 'forced_eos_token_id': 0}
